<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5> Coursera C09 W04 - Creative Use of FourSquare</font></h1>

<h1 align=center><font size = 5> Coursera C09 W04 - Real Estate Analysis - São Paulo City - Brazil</font></h1>

## Introduction

This Lab Session is part of the **Coursera-IBM course "Applied Data Science Capstone"** and it is the task for Week 04.\
Students are asked to devise a Business issue and to propose a creative solution to it by applying tools and knowlwdge acquired thorought the course. Foursquare API usage is a must. 

This Notebook will analyse Real Estate situation in the city of São Paulo (Brazil) from different but not conflicting standpoints.

São Paulo is a big city, a true Metropole, where 15 million people live and around 25 million people work. People who wants to live in São Paulo all have different needs and perspectives when it comes to acquire a place to live, be it an apartment, be it a house.

Their choices depend on several factors:\
a) Professional career moment and Age (newly-graduated or seasoned professional or retired);\
b) Family structure (single, newly-married, married without children, married with children or divorced);\
c) Power of purchasing (low income, high income, combined income);\
d) Resale value, which also depends on items 'a' and 'b';\
e) And, **what is the driver for this analysis**, what is around their place of choice, that is, what does the neighbourhood offers to make their life bearable (supermarkets, gym, subway transportation, bus transportation, shopping malls, etc).

An interesting aspect of this study lies on its double utility => what might play an essential role for a prospective buyer to acquire his-her "home, sweet home" might also be a powerful resource for a good Real Estate salesperson if he-she reads the customer accurately and have data to endorse his-her portfolio.

## Some few lines about São Paulo (Sampa):

In a city as big and diverse as São Paulo, it is not a surprise that people are also diverse in their preferences. The same thing can be said about real estate offerings. However, in order to keep this task´s dimensions compatible to the purpose of concepts learning and experimentation, I will restrain it to 2 main types of real estate. 

By the way, this is not "a jump in the dark" at all because recently I was interested in acquiring an Studio for investiment purposes and spent some time talking to salesperson in their showrooms. From these conversations, I realized that every real estate enterprise will try to offer what they call "special features" in the hope they match customers´ needs. As diverse as they may be, it is possible to bring them down to only two:

**1) Studio:** It really looks like a hotel room with some added accessorioes. The area ranges from 25 to 33 square meters.\
**2) Regular:** Tt is your "regular" apartment everybody is used to => 2, 3 or 4 bedrooms (sometimes with exclusive bathrooms), living room, kitchen, dining room, etc. The area ranges from 50 to 180 square meter. 

In a city that never ceases to grow, growth means going upwards, means tall buildings and skyscrappers. Supply and Demand for houses are proportionaly low because of security issues. Urban violence scares every São Paulo inhabitant.

Potential Real Estate buyers can also be dwarfed to three categories:\
**1) Studio Buyer:** He-she is usually a single young person starting a career and who has different values from his-her parents. A car is undesirable and Uber or Public Transportation are prefered. Technology-addicted with a taste for cosmopolitan cuisines.\
**2) Regular Buyer:** For this case, being married is the most common marital status and the couple wants a place to live for several years and start a family. They are people that could once have been Studio buyers but also people from neighbor towns that want spend less time commuting. Car (or cars) is a statement of social position as a good Health plan.\
**3) Investor Buyer:** This buyer is the most senior of all. It is common in Brazil people who retire but still need to have income. Some of these people acquire apartments (and houses) to rent. They might acquire any type of Real State, Studios os Regulars, in any combination they think will maximize their income.

## Methodology and Professional Target

**Professionals targeted:** Taking into account everything that was briefly explained above, the underlying idea for this Notebook lies on providing **potential buyers and also real state professionals** a index-based tool to help them identifying their targets and making the best of business opportunities.

**Methodology:** \
a) Selection of 25 São Paulo boroughs which are known for decades as the best locations in the city for real estate investment;\
b) Extraction from FourSquare of all interesting venues around each of these boroughs;\
c) Further Classification of borough venues, which will feature concatenation of similar ones into a wider one (Bar and Pub turn into Bar);\
d) Development of "Buyer-Drivers-and-Preferences" dataframe which will lock to the Classification labels built on item c);\
e) **"Attractiveness Index"** calculation which will be the multiplication of the number(frequency) of venue occurence by the "power of attraction" it displays for each type of buyer;\
f) Index-based Decision.

</p> A note about "Attractiveness Index" calculation. It is quite simple indeed. A Studio buyer favours Public Transportation, therefore any proximity to Subway stations will be more important to him-her than any Auto-Services facilities around their future "home, sweet home". For Regular buyers it is exactly the opposite and Investor buyers will be some sort of average between the two former types.

**Step 01:**\
Libraries Import

In [1]:
import pandas as pd

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print()
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    openssl-1.1.1i             |       h7f98852_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0

The following packages will be UPDATED:

  openssl                                 1.1.1h-h516909a_0 --> 1.1.1i-h7f98852_0



openssl-1.1.1i       | 2.1 MB    | #####

In [2]:
pip install lxml

     |████████████████████████████████| 5.5MB 22.2MB/s eta 0:00:01     |█████████████████████████▏      | 4.4MB 22.2MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [28]:
# !wget -O sao_paulo_boroughs.xlsx https://github.com/Alexandre-Neri/Coursera_Capstone/blob/master/sao_paulo_boroughs.xlsx

!wget -O sao_paulo_boroughs.csv https://github.com/Alexandre-Neri/Coursera_Capstone/blob/master/sao_paulo_boroughs.csv

--2020-12-13 16:28:20--  https://github.com/Alexandre-Neri/Coursera_Capstone/blob/master/sao_paulo_boroughs.csv
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘sao_paulo_boroughs.csv’

sao_paulo_boroughs.     [ <=>                ]  92.15K  --.-KB/s    in 0.06s   

2020-12-13 16:28:21 (1.40 MB/s) - ‘sao_paulo_boroughs.csv’ saved [94357]



In [33]:
#subdivisions = pd.read_csv("sao_paulo_boroughs.csv")

In [34]:
#subdivisions = pd.read_excel("sao_paulo_boroughs.xls")

**Step 02:**\
São Paulo Buroughs Data Import

In [94]:
subdivisions = pd.read_excel("sao_paulo_boroughs.xlsx")

In [95]:
subdivisions

,Borough,Latitude,Longitude,City_Sector
0,Bela Vista,-23.549112,-46.652273,Downtown
1,Higienopolis,-23.543747,-46.656409,Downtown
2,Mooca,-23.564321,-46.596572,Downtown
3,Santa Cecilia,-23.534434,-46.655584,Downtown
4,Analia Franco,-23.561479,-46.563194,East
5,Tatuape,-23.545035,-46.573325,East
6,Santana,-23.502883,-46.624897,North
7,Brooklyn,-23.626459,-46.687502,South
8,Campo Belo,-23.622684,-46.671107,South
9,Ipiranga,-23.588708,-46.610538,South


**Step 03:**\
FourSquare API Credentials and Extraction of Venues Data

In [96]:
CLIENT_ID = '113ZWQSUYLUGMKP51IRZVZRVDHJBACXCYTXIXGD5GM5PVO5B' # your Foursquare ID
CLIENT_SECRET = '5QXPGLDYT1SVGN3FVVLMZEV0EKJMYPMJNMJAMF2X3ZZEW0SP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 350 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 113ZWQSUYLUGMKP51IRZVZRVDHJBACXCYTXIXGD5GM5PVO5B
CLIENT_SECRET:5QXPGLDYT1SVGN3FVVLMZEV0EKJMYPMJNMJAMF2X3ZZEW0SP


In [97]:
# From Google, São Paulo city geographical coordinates are:
sampa_lat = -23.5474
sampa_lng = -46.63611

In [98]:
radius = 5000
place = ''

In [99]:
# define the dataframe columns
column_names = ['Borough','Venue_Name', 'Business_Category','Venue_Latitude', 'Venue_Longitude', 'Decision'] 

# instantiate the dataframe
sp_df = pd.DataFrame(columns=column_names)

In [100]:
sp_df

,Borough,Venue_Name,Business_Category,Venue_Latitude,Venue_Longitude,Decision


In [101]:
# This module turns 'request' output into a pandas df, so that we can work with data and get insights
# Key concept here lies on understading the nature of how every piece of information, that is, whether part of a list or a dictionary
# and how to correctly extract them in order to build a functional dataframe.

for i in range(len(subdivisions['Borough'])):
    sp_lat = subdivisions['Latitude'][i]
    sp_lng = subdivisions['Longitude'][i]
    print("The name of the burough is {} with coordinates => Latitude {} and Longitude {}"
          .format(subdivisions['Borough'][i], subdivisions['Latitude'][i], subdivisions['Longitude'][i]))
    radius = 4000
    place = ''
    
    url_sp = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&query={}'.format(CLIENT_ID, CLIENT_SECRET, sp_lat, sp_lng, VERSION, radius, LIMIT,place)
    
    results_sp = requests.get(url_sp).json()
    
    sp_data = results_sp['response']['venues']
        
    for count in sp_data: 
#     print(count)
        Name = count['name'] 
    
        if len(count['categories']) == 0:
            BizzCat = 'NaN'
        else:
            temp = list(count['categories'][0].values())   # from dict 'categories', extract the values, turn them into a list to pick pos '0'
            BizzCat = temp[1]
    
        tmp = count['location']
        chv = list(tmp.keys())
#     print('chv zero is', chv[0], 'for name =>', Name)    
    
        if chv[0] == 'address':
            if chv[1] == 'crossStreet':
                sp_latlng = list(tmp.values())
                Latitude = sp_latlng[2]
                Longitude = sp_latlng[3]
            else:
                sp_latlng = list(tmp.values())
                Latitude = sp_latlng[1]
                Longitude = sp_latlng[2]
        else:
            if chv[0] == 'crossStreet':
                sp_latlng = list(tmp.values())
                Latitude = sp_latlng[1]
                Longitude = sp_latlng[2]
            else:
                sp_latlng = list(tmp.values())
                Latitude = sp_latlng[0]
                Longitude = sp_latlng[1]
        
#     print('=>')
        borough = subdivisions['Borough'][i]
      
        sp_df = sp_df.append({'Borough':borough, 'Venue_Name': Name, 'Business_Category':BizzCat, 'Venue_Latitude': Latitude, 'Venue_Longitude': Longitude}, ignore_index=True)

The name of the burough is Bela Vista with coordinates => Latitude -23.549112 and Longitude -46.652273
The name of the burough is Higienopolis with coordinates => Latitude -23.543747 and Longitude -46.656409
The name of the burough is Mooca with coordinates => Latitude -23.564321 and Longitude -46.596572
The name of the burough is Santa Cecilia with coordinates => Latitude -23.5344342 and Longitude -46.655584
The name of the burough is Analia Franco with coordinates => Latitude -23.561479 and Longitude -46.563194
The name of the burough is Tatuape with coordinates => Latitude -23.545035 and Longitude -46.573325
The name of the burough is Santana with coordinates => Latitude -23.502883 and Longitude -46.624897
The name of the burough is Brooklyn with coordinates => Latitude -23.626459 and Longitude -46.687502
The name of the burough is Campo Belo with coordinates => Latitude -23.622684 and Longitude -46.671107
The name of the burough is Ipiranga with coordinates => Latitude -23.588708 a

**Step 04:**\
Data Cleansing:\
a) Column 'Decison' will help eliminate redundant or unimportant Business Categories (Yes=Remain - No=Discard);\
b) Unfortunately, data in Brazil are rare and very poor and some Venue Names do not have any Business Category description. Therefore, there will be row deletion but a calculation will be performed to ensure the loss of data is not significant;\
c) 'Business Category' concatenation => in order to keep this task within a reasonable computational size, labels will be merged accordingly to their similarities (Hospitals and Medical Centers will be together under 'Medical Services' entry, for example;\

In [102]:
sp_df.head()

,Borough,Venue_Name,Business_Category,Venue_Latitude,Venue_Longitude,Decision
0,Bela Vista,Estação Higienópolis - Mackenzie (Metrô),Metro Station,-23.548846,-46.652371,NaN
1,Bela Vista,Palácio Paralelo,Candy Store,-23.549162,-46.652235,NaN
2,Bela Vista,Santa Gula,Restaurant,-23.547085,-46.649708,NaN
3,Bela Vista,Juzanseverini's,Assisted Living,-23.549173,-46.652009,NaN
4,Bela Vista,Estação Luz (CPTM),Train Station,-23.536232,-46.634527,NaN


In [103]:
before = sp_df.shape
before

(2911, 6)

In [104]:
sp_df = sp_df[sp_df.Business_Category != 'NaN'].reset_index()
sp_df = sp_df.drop('index', axis=1)
sp_df.head()

,Borough,Venue_Name,Business_Category,Venue_Latitude,Venue_Longitude,Decision
0,Bela Vista,Estação Higienópolis - Mackenzie (Metrô),Metro Station,-23.548846,-46.652371,NaN
1,Bela Vista,Palácio Paralelo,Candy Store,-23.549162,-46.652235,NaN
2,Bela Vista,Santa Gula,Restaurant,-23.547085,-46.649708,NaN
3,Bela Vista,Juzanseverini's,Assisted Living,-23.549173,-46.652009,NaN
4,Bela Vista,Estação Luz (CPTM),Train Station,-23.536232,-46.634527,NaN


In [105]:
after = sp_df.shape
after

(2856, 6)

In [107]:
print("Loss of data is", before[0]-after[0], "rows or", (((before[0]-after[0])/before[0]))*100,"%".)

Loss of data is 55 rows or 1.8893850910340089 %


In [108]:
sp_df.tail()

,Borough,Venue_Name,Business_Category,Venue_Latitude,Venue_Longitude,Decision
2851,Vila Leopoldina,Centro de Acolhida para Adultos Zancone,Building,-23.532618,-46.729003,NaN
2852,Vila Leopoldina,Integra Odontologia Moderna,Dentist's Office,-23.531178,-46.729324,NaN
2853,Vila Leopoldina,Odontologia Moura Campos,Dentist's Office,-23.528918,-46.730721,NaN
2854,Vila Leopoldina,Dr. Jorge Naomi,Dentist's Office,-23.531898,-46.730898,NaN
2855,Vila Leopoldina,Stampart Gráfica Especial,Factory,-23.531033,-46.730473,NaN


In [109]:
for j in range(sp_df.shape[0]):
    if sp_df['Business_Category'][j] == 'Office' or sp_df['Business_Category'][j] == 'Building' or sp_df['Business_Category'][j] == 'Business Center' or sp_df['Business_Category'][j] == 'Tech Startup':
        sp_df['Business_Category'][j] = 'Office'
        sp_df['Decision'][j] = 'Yes'
    elif sp_df['Business_Category'][j] == 'Gym' or sp_df['Business_Category'][j] == 'Gym / Fitness Center' or sp_df['Business_Category'][j] == 'Athletics & Sports' or sp_df['Business_Category'][j] == 'Gymnastics Gym':
        sp_df['Business_Category'][j] = 'Gym'
        sp_df['Decision'][j] = 'Yes'
    elif sp_df['Business_Category'][j] == 'Coffee Shop' or sp_df['Business_Category'][j] == 'Café' or sp_df['Business_Category'][j] == 'Cafeteria' or sp_df['Business_Category'][j] == 'Tea Room' or sp_df['Business_Category'][j] == 'Breakfast Spot':
        sp_df['Business_Category'][j] = 'Coffee Shop'
        sp_df['Decision'][j] = 'Yes'
    elif sp_df['Business_Category'][j] == 'Coworking Space' or sp_df['Business_Category'][j] == 'Meeting Room' or sp_df['Business_Category'][j] == 'Conference Room' or sp_df['Business_Category'][j] == 'Event Space':
        sp_df['Business_Category'][j] = 'Coworking Space'
        sp_df['Decision'][j] = 'Yes'
    elif sp_df['Business_Category'][j] == 'Church' or sp_df['Business_Category'][j] == 'Temple' or sp_df['Business_Category'][j] == 'Spiritual Center':
        sp_df['Business_Category'][j] = 'Church & Temple'
        sp_df['Decision'][j] = 'Yes'
    elif sp_df['Business_Category'][j] == 'Bus Stop' or sp_df['Business_Category'][j] == 'Bus Station':
        sp_df['Business_Category'][j] = 'Bus Stop'
        sp_df['Decision'][j] = 'Yes'
    elif sp_df['Business_Category'][j] == 'Medical Center' or sp_df['Business_Category'][j] == 'Medical Lab' or sp_df['Business_Category'][j] == 'Hospital':
        sp_df['Business_Category'][j] = 'Medical Services (Hospitals & Laboratories)'
        sp_df['Decision'][j] = 'Yes'
    elif sp_df['Business_Category'][j] == 'Automotive Shop' or sp_df['Business_Category'][j] == 'Auto Garage' or sp_df['Business_Category'][j] == 'Auto Dealership' or sp_df['Business_Category'][j] == 'Car Wash' or sp_df['Business_Category'][j] == 'Auto Workshop':
        sp_df['Business_Category'][j] = 'Automotive Services (Sales & Maintenance)'
        sp_df['Decision'][j] = 'Yes'
    elif sp_df['Business_Category'][j] == 'Supermarket' or sp_df['Business_Category'][j] == 'Market' or sp_df['Business_Category'][j] == 'Grocery Store' or sp_df['Business_Category'][j] == 'Butcher':
        sp_df['Business_Category'][j] = 'Supermarket'
        sp_df['Decision'][j] = 'Yes'
    elif sp_df['Business_Category'][j] == 'Restaurant' or sp_df['Business_Category'][j] == 'Brazilian Restaurant' or sp_df['Business_Category'][j] == 'Italian Restaurant' or sp_df['Business_Category'][j] == 'Asian Restaurant' or sp_df['Business_Category'][j] == 'Diner' or sp_df['Business_Category'][j] == 'Pizza Place' or sp_df['Business_Category'][j] == 'Chinese Restaurant' or sp_df['Business_Category'][j] == 'Japanese Restaurant' or sp_df['Business_Category'][j] == 'Sushi Restaurant' or sp_df['Business_Category'][j] == 'Middle Eastern Restaurant':
        sp_df['Business_Category'][j] = 'Restaurant'
        sp_df['Decision'][j] = 'Yes'
    elif sp_df['Business_Category'][j] == 'Snack Place' or sp_df['Business_Category'][j] == 'Food Truck' or sp_df['Business_Category'][j] == 'Candy Store' or sp_df['Business_Category'][j] == 'Pastelaria' or sp_df['Business_Category'][j] == 'Food Stand' or sp_df['Business_Category'][j] == 'Burger Joint' or sp_df['Business_Category'][j] == 'Hot Dog Joint':
        sp_df['Business_Category'][j] = 'Snack Place'
        sp_df['Decision'][j] = 'Yes'
    elif sp_df['Business_Category'][j] == 'Health & Beauty Service' or sp_df['Business_Category'][j] == 'Nail Salon' or sp_df['Business_Category'][j] == 'Spa' or sp_df['Business_Category'][j] == 'Boutique' or sp_df['Business_Category'][j] == 'Cosmetics Shop':
        sp_df['Business_Category'][j] = 'Women´s Personnal Care'
        sp_df['Decision'][j] = 'Yes'
    elif sp_df['Business_Category'][j] == 'Electronics Store' or sp_df['Business_Category'][j] == 'Mobile Phone Shop' or sp_df['Business_Category'][j] == 'IT Services':
        sp_df['Business_Category'][j] = 'Electronics & Mobile'
        sp_df['Decision'][j] = 'Yes'
    elif sp_df['Business_Category'][j] == 'High School' or sp_df['Business_Category'][j] == 'College Lab' or sp_df['Business_Category'][j] == 'University' or sp_df['Business_Category'][j] == 'Student Center' or sp_df['Business_Category'][j] == 'College Auditorium' or sp_df['Business_Category'][j] == 'General College & University' or sp_df['Business_Category'][j] == 'School' or sp_df['Business_Category'][j] == 'Language School':
        sp_df['Business_Category'][j] = 'Educational Services'
        sp_df['Decision'][j] = 'Yes'
    elif sp_df['Business_Category'][j] == 'Bar' or sp_df['Business_Category'][j] == 'Pub' or sp_df['Business_Category'][j] == 'Internet Café':
        sp_df['Business_Category'][j] = 'Bar'
        sp_df['Decision'][j] = 'Yes'
    elif sp_df['Business_Category'][j] == 'Salon / Barbershop' or sp_df['Business_Category'][j] == 'Men´s Store' or sp_df['Business_Category'][j] == 'Sporting Goods Shop':
        sp_df['Business_Category'][j] = 'Men´s Personnal Care'
        sp_df['Decision'][j] = 'Yes'
    elif sp_df['Business_Category'][j] == 'Shoe Store' or sp_df['Business_Category'][j] == 'Shoe Repair' or sp_df['Business_Category'][j] == 'Department Store' or sp_df['Business_Category'][j] == 'Clothing Store' or sp_df['Business_Category'][j] == 'Miscellaneous Shop':
        sp_df['Business_Category'][j] = 'Clothing & Shoes'
        sp_df['Decision'][j] = 'Yes'
    elif sp_df['Business_Category'][j] == 'Residential Building (Apartment / Condo)':
        sp_df['Decision'][j] = 'Yes'
    elif sp_df['Business_Category'][j] == "Doctor's Office":
        sp_df['Decision'][j] = 'Yes'
    elif sp_df['Business_Category'][j] == "Dentist's Office":
        sp_df['Decision'][j] = 'Yes'
    elif sp_df['Business_Category'][j] == 'Bank':
        sp_df['Decision'][j] = 'Yes'
    elif sp_df['Business_Category'][j] == 'Bus Line':
        sp_df['Decision'][j] = 'Yes'
    elif sp_df['Business_Category'][j] == 'Pharmacy':
        sp_df['Decision'][j] = 'Yes'
    elif sp_df['Business_Category'][j] == 'Gas Station':
        sp_df['Decision'][j] = 'Yes'
    elif sp_df['Business_Category'][j] == 'Metro Station':
        sp_df['Decision'][j] = 'Yes'
    elif sp_df['Business_Category'][j] == 'Train Station':
        sp_df['Decision'][j] = 'Yes'
    elif sp_df['Business_Category'][j] == 'Laundry Service':
        sp_df['Decision'][j] = 'Yes'
    else:
        sp_df['Decision'][j] = 'No'

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

In [110]:
sp_df['Business_Category'].value_counts().to_frame()

,Business_Category
Office,454
Residential Building (Apartment / Condo),125
Restaurant,117
Snack Place,99
Doctor's Office,89
Women´s Personnal Care,87
Men´s Personnal Care,87
Clothing & Shoes,85
Automotive Services (Sales & Maintenance),72
Dentist's Office,72


In [111]:
sp_df.head()

,Borough,Venue_Name,Business_Category,Venue_Latitude,Venue_Longitude,Decision
0,Bela Vista,Estação Higienópolis - Mackenzie (Metrô),Metro Station,-23.548846,-46.652371,Yes
1,Bela Vista,Palácio Paralelo,Snack Place,-23.549162,-46.652235,Yes
2,Bela Vista,Santa Gula,Restaurant,-23.547085,-46.649708,Yes
3,Bela Vista,Juzanseverini's,Assisted Living,-23.549173,-46.652009,No
4,Bela Vista,Estação Luz (CPTM),Train Station,-23.536232,-46.634527,Yes


In [112]:
sp_df = sp_df[sp_df.Decision != 'No'].reset_index()
sp_df = sp_df.drop('index', axis=1)
sp_df.head()

,Borough,Venue_Name,Business_Category,Venue_Latitude,Venue_Longitude,Decision
0,Bela Vista,Estação Higienópolis - Mackenzie (Metrô),Metro Station,-23.548846,-46.652371,Yes
1,Bela Vista,Palácio Paralelo,Snack Place,-23.549162,-46.652235,Yes
2,Bela Vista,Santa Gula,Restaurant,-23.547085,-46.649708,Yes
3,Bela Vista,Estação Luz (CPTM),Train Station,-23.536232,-46.634527,Yes
4,Bela Vista,Edifício Consolata,Residential Building (Apartment / Condo),-23.549069,-46.652204,Yes


In [113]:
sp_df.shape

(1922, 6)

In [114]:
# drops columns no longer useful
new_sp_df = sp_df.drop('Decision', 1)
new_sp_df.head()

,Borough,Venue_Name,Business_Category,Venue_Latitude,Venue_Longitude
0,Bela Vista,Estação Higienópolis - Mackenzie (Metrô),Metro Station,-23.548846,-46.652371
1,Bela Vista,Palácio Paralelo,Snack Place,-23.549162,-46.652235
2,Bela Vista,Santa Gula,Restaurant,-23.547085,-46.649708
3,Bela Vista,Estação Luz (CPTM),Train Station,-23.536232,-46.634527
4,Bela Vista,Edifício Consolata,Residential Building (Apartment / Condo),-23.549069,-46.652204


**Step 05:**\
Frequency of Venue Type Occurence Calculation

In [72]:
sampa_onehot = pd.get_dummies(new_sp_df[['Business_Category']], prefix="", prefix_sep="")
sampa_onehot['Borough'] = new_sp_df['Borough'] 

fixed_columns = [sampa_onehot.columns[-1]] + list(sampa_onehot.columns[:-1]) #establishes column sequence(order)
sampa_onehot = sampa_onehot[fixed_columns]

sampa_grouped = sampa_onehot.groupby('Borough').mean().reset_index()

sampa_grouped.head(25)

,Borough,Automotive Services (Sales & Maintenance),Bank,Bar,Bus Line,Bus Stop,Church & Temple,Clothing & Shoes,Coffee Shop,Coworking Space,Dentist's Office,Doctor's Office,Educational Services,Electronics & Mobile,Gas Station,Gym,Laundry Service,Medical Services (Hospitals & Laboratories),Men´s Personnal Care,Metro Station,Office,Pharmacy,Residential Building (Apartment / Condo),Restaurant,Snack Place,Supermarket,Train Station,Women´s Personnal Care
0,Analia Franco,0.028571,0.071429,0.000000,0.014286,0.014286,0.014286,0.242857,0.042857,0.000000,0.014286,0.028571,0.100000,0.057143,0.000000,0.028571,0.000000,0.071429,0.057143,0.000000,0.028571,0.000000,0.028571,0.000000,0.028571,0.014286,0.000000,0.114286
1,Barra Funda,0.027778,0.055556,0.013889,0.055556,0.013889,0.000000,0.013889,0.027778,0.083333,0.000000,0.013889,0.000000,0.027778,0.013889,0.000000,0.000000,0.013889,0.041667,0.000000,0.472222,0.000000,0.013889,0.083333,0.027778,0.000000,0.000000,0.000000
2,Bela Vista,0.076923,0.000000,0.076923,0.000000,0.000000,0.000000,0.019231,0.019231,0.019231,0.019231,0.038462,0.076923,0.000000,0.000000,0.019231,0.019231,0.038462,0.038462,0.019231,0.153846,0.000000,0.173077,0.115385,0.057692,0.000000,0.019231,0.000000
3,Brooklyn,0.023256,0.034884,0.011628,0.023256,0.081395,0.000000,0.011628,0.011628,0.034884,0.011628,0.023256,0.011628,0.000000,0.023256,0.023256,0.000000,0.011628,0.058140,0.011628,0.441860,0.011628,0.058140,0.023256,0.023256,0.011628,0.000000,0.023256
4,Butanta,0.034884,0.069767,0.011628,0.151163,0.104651,0.000000,0.011628,0.081395,0.011628,0.046512,0.023256,0.011628,0.023256,0.000000,0.000000,0.000000,0.046512,0.011628,0.023256,0.104651,0.046512,0.000000,0.046512,0.093023,0.011628,0.011628,0.023256
5,Campo Belo,0.051282,0.012821,0.025641,0.000000,0.000000,0.000000,0.038462,0.000000,0.025641,0.025641,0.102564,0.025641,0.012821,0.012821,0.051282,0.025641,0.012821,0.051282,0.000000,0.192308,0.025641,0.025641,0.153846,0.012821,0.025641,0.000000,0.089744
6,Cerqueira Cesar,0.000000,0.000000,0.000000,0.000000,0.012987,0.012987,0.142857,0.051948,0.051948,0.025974,0.038961,0.038961,0.012987,0.000000,0.025974,0.000000,0.000000,0.168831,0.000000,0.116883,0.012987,0.025974,0.077922,0.077922,0.000000,0.000000,0.103896
7,Higienopolis,0.000000,0.085106,0.021277,0.021277,0.042553,0.000000,0.042553,0.106383,0.021277,0.042553,0.063830,0.063830,0.000000,0.021277,0.000000,0.000000,0.000000,0.063830,0.000000,0.106383,0.021277,0.148936,0.063830,0.021277,0.000000,0.000000,0.042553
8,Ipiranga,0.172414,0.034483,0.034483,0.000000,0.000000,0.034483,0.000000,0.017241,0.068966,0.000000,0.000000,0.155172,0.000000,0.017241,0.017241,0.000000,0.034483,0.000000,0.000000,0.206897,0.000000,0.034483,0.051724,0.086207,0.000000,0.000000,0.034483
9,Lapa,0.047059,0.023529,0.000000,0.000000,0.011765,0.023529,0.035294,0.023529,0.035294,0.070588,0.058824,0.023529,0.023529,0.023529,0.058824,0.058824,0.035294,0.094118,0.000000,0.117647,0.023529,0.035294,0.035294,0.023529,0.011765,0.000000,0.105882


In [115]:
sampa_grouped.shape

(25, 28)

**Step 06:**\
Prospective Buyer´s Preference and Drivers Dataframe Import

In [116]:
weights = pd.read_excel("sp_attraction_values.xlsx")
weights.head()

,Buyer Type,Automotive Services (Sales & Maintenance),Bank,Bar,Bus Line,Bus Stop,Church & Temple,Clothing & Shoes,Coffee Shop,Coworking Space,Dentist's Office,Doctor's Office,Educational Services,Electronics & Mobile,Gas Station,Gym,Laundry Service,Medical Services (Hospitals & Laboratories),Men´s Personnal Care,Metro Station,Office,Pharmacy,Residential Building (Apartment / Condo),Restaurant,Snack Place,Supermarket,Train Station,Women´s Personnal Care
0,Studio Buyer,0.00,0.00,0.02,0.05,0.05,0.02,0.040,0.05,0.05,0.02,0.02,0.05,0.05,0.00,0.05,0.05,0.05,0.02,0.10,0.10,0.02,0.00,0.05,0.02,0.05,0.05,0.02
1,Regular Buyer,0.10,0.04,0.00,0.00,0.00,0.08,0.007,0.03,0.03,0.03,0.04,0.02,0.06,0.10,0.00,0.01,0.10,0.02,0.00,0.08,0.07,0.01,0.06,0.02,0.07,0.00,0.02
2,Investor Buyer,0.03,0.01,0.01,0.04,0.04,0.05,0.020,0.03,0.04,0.04,0.03,0.05,0.05,0.03,0.03,0.03,0.06,0.02,0.07,0.09,0.04,0.03,0.05,0.02,0.04,0.03,0.02


In [117]:
weights.shape[0]

3

**Step 07:**\
Final Dataframe Definition **(Attractiveness Indexes)** and Software-Oriented Decision

In [83]:
# define the dataframe columns
column_names = ['Borough','Attraction_Value_Studio', 'Attraction_Value_Regular','Attraction_Value_Investor'] 

# instantiate the dataframe
sp_attract_df = pd.DataFrame(columns=column_names)

In [84]:
sp_attract_df.head()

,Borough,Attraction_Value_Studio,Attraction_Value_Regular,Attraction_Value_Investor


In [85]:
final_sp = sampa_grouped.drop('Borough', 1)
final_buyer = weights.drop('Buyer Type', 1)

In [86]:
for i in range(sampa_grouped.shape[0]):
    j = 0
    sp_attract_df['Borough'] = sampa_grouped['Borough']
    sp_attract_df['Attraction_Value_Studio'][i] = sum(final_sp.iloc[i] * final_buyer.iloc[j])*1000
    sp_attract_df['Attraction_Value_Regular'][i] = sum(final_sp.iloc[i] * final_buyer.iloc[j+1])*1000
    sp_attract_df['Attraction_Value_Investor'][i] = sum(final_sp.iloc[i] * final_buyer.iloc[j+2])*1000
sp_attract_df

,Borough,Attraction_Value_Studio,Attraction_Value_Regular,Attraction_Value_Investor
0,Analia Franco,34.8571,31.5571,32
1,Barra Funda,65,57.7361,60.2778
2,Bela Vista,39.0385,39.5577,42.6923
3,Brooklyn,60.6977,51.2442,57.6744
4,Butanta,43.3721,35.5465,40.9302
5,Campo Belo,44.1026,47.1923,44.1026
6,Cerqueira Cesar,39.8701,31.2597,35.974
7,Higienopolis,33.8298,32.6383,36.5957
8,Ipiranga,41.7241,54.6552,46.3793
9,Lapa,37.0588,41.1882,38.7059


In [87]:
print()
print("FINAL CONCLUSIONS ABOUT REAL-ESTATE STUDY IN THE CITY OF SÃO PAULO - BRAZIL")
print()
bestStudio = sp_attract_df.sort_values('Attraction_Value_Studio', ascending=False).reset_index(drop=True)
print("The best Borough in São Paulo for Studio buyers is {} with an Attractiveness Index of {}".format(bestStudio['Borough'][0], bestStudio['Attraction_Value_Studio'][0]))
print()
bestRegular = bestStudio.sort_values('Attraction_Value_Regular', ascending=False).reset_index(drop=True)
print("The best Borough in São Paulo for Regular buyers is {} with an Attractiveness Index of {}".format(bestStudio['Borough'][0], bestStudio['Attraction_Value_Regular'][0]))
print()
bestInvestor = bestRegular.sort_values('Attraction_Value_Investor', ascending=False).reset_index(drop=True)
print("The best Borough in São Paulo for Investor buyers is {} with an Attractiveness Index of {}".format(bestRegular['Borough'][0], bestRegular['Attraction_Value_Investor'][0]))
print()


FINAL CONCLUSIONS ABOUT REAL-ESTATE STUDY IN THE CITY OF SÃO PAULO - BRAZIL

The best Borough in São Paulo for Studio buyers is Vila Olimpia with an Attractiveness Index of 76.66666666666666

The best Borough in São Paulo for Regular buyers is Vila Olimpia with an Attractiveness Index of 60.397849462365585

The best Borough in São Paulo for Investor buyers is Vila Olimpia with an Attractiveness Index of 67.84946236559139



**Step 08:**\
Map Visualization

In [88]:
# create map of São Paulo Hill using latitude and longitude values and display Business ventures around it
map_sp = folium.Map(location=[sampa_lat, sampa_lng], zoom_start=20)

# add markers to map
for lat, lng, name, categories in zip(new_sp_df['Venue_Latitude'], new_sp_df['Venue_Longitude'], new_sp_df['Venue_Name'], new_sp_df['Business_Category']):
    label = '{}, {}'.format(name, categories) #what will be shown when user hovers a place
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sp)  
    
map_sp